In [2]:
import os

def count_images_in_folder(folder_path):
    # List of valid image extensions
    valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff', '.webp')

    # Get the list of files in the directory
    files = os.listdir(folder_path)
    
    # Count the number of images
    image_count = sum(1 for file in files if file.lower().endswith(valid_extensions))
    
    return image_count

# Path to your image directory
image_dir = './static/image'

# Get the number of images
num_images = count_images_in_folder(image_dir)

print(f"The number of images in the folder is: {num_images}")


The number of images in the folder is: 2000


In [4]:
import os
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import pandas as pd
import torch
from tqdm import tqdm

# Initialize the model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


In [5]:
device = torch.device("cpu")
model.to(device)

image_dir = './static/image'
batch_size = 8
data = []

# Function to process a batch of images
def process_batch(image_paths):
    images = [Image.open(image_path).convert("RGB") for image_path in image_paths]
    inputs = processor(images=images, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    out = model.generate(**inputs, max_new_tokens=20)
    captions = processor.batch_decode(out, skip_special_tokens=True)
    return captions

# Get list of all image paths
image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]

# Process images in batches
for i in tqdm(range(0, len(image_paths), batch_size)):
    batch_paths = image_paths[i:i+batch_size]
    captions = process_batch(batch_paths)
    for img_path, caption in zip(batch_paths, captions):
        image_id = os.path.basename(img_path)
        data.append({'image_id': image_id, 'caption': caption})

df = pd.DataFrame(data)

csv_path = 'captions.csv'
df.to_csv(csv_path, index=False)

print(f"Captions saved to {csv_path}")


100%|██████████| 250/250 [1:56:18<00:00, 27.91s/it]

Captions saved to captions.csv


In [3]:
import numpy as np
import pandas as pd
import time
import re # for regular expressions
from collections import defaultdict
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import contractions
import string
from IPython.display import display, HTML
import nltk
nltk.download('stopwords')
data = pd.read_csv("static/arxiv_data.csv", header='infer')
print("Total rows : ", len(data))
display(HTML(data.head().to_html()))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vaishnavishinde/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Total rows :  51774


,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic Depth Estimation,"Stereo matching is one of the widely used techniques for inferring depth from\nstereo images owing to its robustness and speed. It has become one of the major\ntopics of research since it finds its applications in autonomous driving,\nrobotic navigation, 3D reconstruction, and many other fields. Finding pixel\ncorrespondences in non-textured, occluded and reflective areas is the major\nchallenge in stereo matching. Recent developments have shown that semantic cues\nfrom image segmentation can be used to improve the results of stereo matching.\nMany deep neural network architectures have been proposed to leverage the\nadvantages of semantic segmentation in stereo matching. This paper aims to give\na comparison among the state of art networks both in terms of accuracy and in\nterms of speed which are of higher importance in real-time applications.","['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Recommendations for Trustworthy Artificial Intelligence in Future Medical Imaging,"The recent advancements in artificial intelligence (AI) combined with the\nextensive amount of data generated by today's clinical systems, has led to the\ndevelopment of imaging AI solutions across the whole value chain of medical\nimaging, including image reconstruction, medical image segmentation,\nimage-based diagnosis and treatment planning. Notwithstanding the successes and\nfuture potential of AI in medical imaging, many stakeholders are concerned of\nthe potential risks and ethical implications of imaging AI solutions, which are\nperceived as complex, opaque, and difficult to comprehend, utilise, and trust\nin critical clinical applications. Despite these concerns and risks, there are\ncurrently no concrete guidelines and best practices for guiding future AI\ndevelopments in medical imaging towards increased trust, safety and adoption.\nTo bridge this gap, this paper introduces a careful selection of guiding\nprinciples drawn from the accumulated experiences, consensus, and best\npractices from five large European projects on AI in Health Imaging. These\nguiding principles are named FUTURE-AI and its building blocks consist of (i)\nFairness, (ii) Universality, (iii) Traceability, (iv) Usability, (v) Robustness\nand (vi) Explainability. In a step-by-step approach, these guidelines are\nfurther translated into a framework of concrete recommendations for specifying,\ndeveloping, evaluating, and deploying technically, clinically and ethically\ntrustworthy AI solutions into clinical practice.","['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions for Semi-supervised Medical Image Segmentation,"In this paper, we proposed a novel mutual consistency network (MC-Net+) to\neffectively exploit the unlabeled hard regions for semi-supervised medical\nimage segmentation. The MC-Net+ model is motivated by the observation that deep\nmodels trained with limited annotations are prone to output highly uncertain\nand easily mis-classified predictions in the ambiguous regions (e.g. adhesive\nedges or thin branches) for the image segmentation task. Leveraging these\nregion-level challenging samples can make the semi-supervised segmentation\nmodel training more effective. Therefore, our proposed MC-Net+ model consists\nof two new designs. First, the model contains one shared encoder and multiple\nsightly different decoders (i.e. using different up-sampling strategies). The\nstatistical discrepancy of multiple decoders' outputs is computed to denote the\nmodel's uncertainty, which indicates the unlabeled hard regions. Second, a new\nmutual consistency constraint is enforced between one decoder's probability\noutput and other decoders' soft pseudo labels. In this way, we minimize the\nmodel's uncertainty during training and force the model to generate invariant\nand low-entropy results in such challenging areas of unlabeled data, in order\nto l

In [4]:
# Preprocessing
def preprocess(text):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    # Text normalization
    expanded_words = []    
    for word in text.split():
      # using contractions.fix to expand the shortened words
      expanded_words.append(contractions.fix(word))  
   
    text = ' '.join(expanded_words)
    # Remove punctuations
    translator = str.maketrans("", "", string.punctuation)
    text = text.translate(translator)
    # Remove non-alphabetic characters
    text = re.sub(r"[^a-zA-Z\s]", "", text)  
    tokens = text.lower().split()
    # Stemming and stop-word removal
    tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
    return tokens

In [5]:
# Inverted index construction
def construct_inverted_index(df):
  dictionary = {} # inverted index
 
  for index, row in df.iterrows():
    try:
      abstract_tokens = preprocess(row.summaries) # convert paper abstracts to tokens
      for token in abstract_tokens:
        if token not in dictionary: # add the document index to the postings of each term in the tokenized text
          dictionary[token] = [index]
        else:
          dictionary[token].append(index)
    except:
      continue
 
  #removing duplicates
  dictionary = {a:set(b) for a, b in dictionary.items()}
 
  return dictionary
inverted_index = construct_inverted_index(data)
print("Size of inverted index ", len(inverted_index))

Size of inverted index  71580


In [6]:
def tokenize_infix_expression(expression):
    return expression.split()
def infix_to_postfix(tokens):
    precedence = {"AND": 2, "NOT": 3, "OR":1 } # set the precedence of operators for postfix expression
    stack = []
    postfix = []
    for token in tokens:
        if token in precedence: # add operands first and then operators
            while stack and precedence.get(stack[-1], 0) >= precedence[token]:
                postfix.append(stack.pop())
            stack.append(token)
        elif token == '(':
            stack.append(token)
        elif token == ')':
            while stack and stack[-1] != '(':
                postfix.append(stack.pop())
            stack.pop()
        else:
            postfix.append(token)
    while stack:
        postfix.append(stack.pop())
    return postfix


In [7]:
def evaluate_postfix(postfix):
    stemmer = PorterStemmer()
    stack = []
    for token in postfix:
        if token == "AND": # take intersection of postings
            set2 = stack.pop()
            set1 = stack.pop()
            result = set1.intersection(set2)
            stack.append(result)
        elif token == "NOT": # finding all documents that are not in the postings list
            set1 = stack.pop()
            set2 = set(range(len(data)))
            result = set2.difference(set1)
            stack.append(result)
        elif token == "OR": # take union of postings
            set1 = stack.pop()
            set2 = stack.pop()
            stack.append(set1.union(set2))  # Convert token to a set
        else: # retrive the posting of the stemmed token
          stack.append(inverted_index.get(stemmer.stem(token), set()))
       
    return stack[0]


In [8]:
infix_expression = "cancer AND detection AND machine AND learning AND NOT lung"
# Tokenize the expression
tokens = tokenize_infix_expression(infix_expression)
# Convert to postfix notation
postfix_expression = infix_to_postfix(tokens)
print(postfix_expression)
# # Evaluate the postfix expression
result = evaluate_postfix(postfix_expression)
titles = []
abstracts = []
for res in list(result)[:10]:
  titles.append(data.iloc[res].titles)
  abstracts.append(data.iloc[res].summaries.replace("\n", " "))
results = pd.DataFrame({"Title": titles, "Abstract": abstracts} )
display(HTML(results.to_html()))

['cancer', 'detection', 'AND', 'machine', 'AND', 'learning', 'AND', 'lung', 'NOT', 'AND']


,Title,Abstract
0,A Rigorous Machine Learning Analysis Pipeline for Biomedical Binary Classification: Application in Pancreatic Cancer Nested Case-control Studies with Implications for Bias Assessments,"Machine learning (ML) offers a collection of powerful approaches for detecting and modeling associations, often applied to data having a large number of features and/or complex associations. Currently, there are many tools to facilitate implementing custom ML analyses (e.g. scikit-learn). Interest is also increasing in automated ML packages, which can make it easier for non-experts to apply ML and have the potential to improve model performance. ML permeates most subfields of biomedical research with varying levels of rigor and correct usage. Tremendous opportunities offered by ML are frequently offset by the challenge of assembling comprehensive analysis pipelines, and the ease of ML misuse. In this work we have laid out and assembled a complete, rigorous ML analysis pipeline focused on binary classification (i.e. case/control prediction), and applied this pipeline to both simulated and real world data. At a high level, this 'automated' but customizable pipeline includes a) exploratory analysis, b) data cleaning and transformation, c) feature selection, d) model training with 9 established ML algorithms, each with hyperparameter optimization, and e) thorough evaluation, including appropriate metrics, statistical analyses, and novel visualizations. This pipeline organizes the many subtle complexities of ML pipeline assembly to illustrate best practices to avoid bias and ensure reproducibility. Additionally, this pipeline is the first to compare established ML algorithms to 'ExSTraCS', a rule-based ML algorithm with the unique capability of interpretably modeling heterogeneous patterns of association. While designed to be widely applicable we apply this pipeline to an epidemiological investigation of established and newly identified risk factors for pancreatic cancer to evaluate how different sources of bias might be handled by ML algorithms."
1,"Transport-based analysis, modeling, and learning from signal and data distributions","Transport-based techniques for signal and data analysis have received increased attention recently. Given their abilities to provide accurate generative models for signal intensities and other data distributions, they have been used in a variety of applications including content-based retrieval, cancer detection, image super-resolution, and statistical machine learning, to name a few, and shown to produce state of the art in several applications. Moreover, the geometric characteristics of transport-related metrics have inspired new kinds of algorithms for interpreting the meaning of data distributions. Here we provide an overview of the mathematical underpinnings of mass transport-related methods, including numerical implementation, as well as a review, with demonstrations, of several applications."
2,labelCloud: A Lightweight Domain-Independent Labeling Tool for 3D Object Detection in Point Clouds,"Within the past decade, the rise of applications based on artificial intelligence (AI) in general and machine learning (ML) in specific has led to many significant contributions within different domains. The applications range from robotics over medical diagnoses up to autonomous driving. However, nearly all applications rely on trained data. In case this data consists of 3D images, it is of utmost importance that the labeling is as accurate as possible to ensure high-quality outcomes of the ML models. Labeling in the 3D space is mostly manual work performed by expert workers, where they draw 3D bounding boxes around target objects the ML model should later automatically identify, e.g., pedestrians for autonomous driving or cancer cells within radiography. While a small range of recent 3D labeling tools exist, they all share three major shortcomings: (i) they are specified for autonomous driving application